In [ ]:
import pandas as pd

# Obtains the file and cleans it
FILE_PATH = 'data/Advertising.csv' 
df = pd.read_csv(FILE_PATH).drop('Unnamed: 0', axis=1)

# Obtains our x and y vectors
x = df.TV.to_numpy()
y = df.Sales.to_numpy()

# Displays the dataframe
df

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class KL_Divergence(nn.module):
    def __init__(self, rand_size=100):
        # The parameters needed to do divergence
        self.mu_m = nn.Parameter(torch.tensor(1.0), requires_grad=True)
        self.sig_m = nn.Parameter(torch.tensor(1.0), rquires_grad=True)
        self.mu_b = nn.Parameter(torch.tensor(1.0), requires_grad=True)
        self.sig_b = nn.Parameter(torch.tensor(1.0), require_grad=True)
        self.noise = nn.Parameter(torch.tensor(1.0), require_grad=True)
        
        # The size parameter used for random generation
        self.rand_size = rand_size
        
    def forward(self, x, y):
        # Randomly generated slope and intercept
        m = torch.normal(self.mu_m, self.sig_m, self.rand_size)
        b = torch.normal(self.mu_b, self.sig_b, self.rand_size)
        
        # Gets a list of values to sum
        res = [self.log_q(m[j], b[j]) - self.log_p(m[j], b[j], x, y) for j in range(len(m))]
        
        # Returns the sum of the values
        return torch.sum(res)
    
        
    def log_q(self, m_j, b_j):
        # Calculates the log_q for each m_j and b_j
        log_m = -((m_j - self.mu_m) ** 2 / 2 * self.sig_m ** 2 + 0.5 * torch.log(self.sig_m ** 2))
        log_b = -((b_j - self.mu_b) ** 2 / 2 * self.sig_b ** 2 + 0.5 * torch.log(self.sig_b ** 2))
        return log_m + log_b
    
    def log_p(self, m_j, b_j, x, y):
        # Calculates the log_p for each m_j and b_j
        vals = [(y[i] - x[i] * m_j - b_j) ** 2 / 2 * self.noise ** 2 for i in range(len(x))]
        return torch.log(torch.sum(vals))
    
    # Trains the specific model
    def train(x, y, model=KL_Divergence(), epochs=5, frac=0.1):
        optimizer = optim.Adam(model.parameters())
        
        for epoch in tqdm(range(epochs), desc="Training..."):
            # Zeros gradiant for training
            optimizer.zero_grad()
            
            # Random selection of data points per iteration
            indices = np.random.randint(low=0, high=len(x), size=int(len(x) * frac))
            
            # Calculates likelihood
            loglik = model(x[indices], y[indices])
            #loglik = model(x, y)
            
            # Mean of loglik
            e = -torch.mean(loglik)
            
            # Updates parameters
            e.backward()
            optimizer.step()
            
        return model

IndentationError: expected an indented block (2706030897.py, line 21)